In [1]:
import numpy as np
import scipy.linalg as linalg
import matplotlib.pyplot as plt
from py_vollib import black_scholes_merton as bsm

# Black Scholes Formula
$
{\frac {\partial V}{\partial t}}+{\frac {1}{2}}\sigma ^{2}S^{2}{\frac {\partial ^{2}V}{\partial S^{2}}}+rS{\frac {\partial V}{\partial S}}-rV=0
$

# Finite Difference Class
## Discretization
- S is discretized by N Steps, $\Delta s = S_{max}/M$, then it can be splited as $\{s_0, s_1, s_2, \ldots, s_M\}$ or $\{0\Delta s, 1\Delta s, 2\Delta s, \ldots, (M-1)\Delta s, M\Delta s\}$
- T is discretized by M Steps, $\Delta t = T/N$, then it can be splited as $\{t_0, t_1, t_2, \ldots, t_M\}$ or $\{0\Delta t, 1\Delta t, 2\Delta t, \ldots, (N-1)\Delta t, N\Delta t\}$
- Using i to represent each space(price) step, $\{i_0, i_1, i_2, \ldots, j_{M-1}, i_M\}$
- Using j to represent each time step: $\{j_0, j_1, j_2, \ldots, j_{N-1}, j_N\}$

In [2]:
class FiniteDifferences(object):

    def __init__(self, S0, K, r, T, sigma, Smax, M, N,
                 is_call=True):
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.sigma = sigma
        self.Smax = Smax
        self.M, self.N = int(M), int(N)  # Ensure M&N are integers
        self.is_call = is_call
        
        # calculate the discretization size of the space(price) and time
        self.dS = Smax / float(self.M)
        self.dt = T / float(self.N)
        
        # create index list
        self.i_values = np.arange(self.M)
        self.j_values = np.arange(self.N)
        
        # save all the option price at each step into a 2-D matrix
        self.grid = np.zeros(shape=(self.M+1, self.N+1))
        
        # manipulate the asset price when the discretize step
        self.boundary_conds = np.linspace(0, Smax, self.M+1)

    def _setup_boundary_conditions_(self):
        pass

    def _setup_coefficients_(self):
        pass

    def _traverse_grid_(self):
        """  Iterate the grid forward in time """
        pass

    def _interpolate_(self):
        """
        Use piecewise linear interpolation on the initial
        grid column to get the closest price at S0.
        """
        return np.interp(self.S0, 
                         self.boundary_conds,
                         self.grid[:, -1])
        # return np.interp(self.S0, 
        #                  self.boundary_conds,
        #                  self.grid[:, 0])
        
    def price(self):
        self._setup_boundary_conditions_()
        self._setup_coefficients_()
        self._traverse_grid_()
        return self._interpolate_()
    
    def print_grid(self):
        """
        get all information through the calculation with structure N*M
        
        The information is stored in a dictionary, with the time step as
        the key and option price range list as the value
        """
        # possible_prices = {}
        # for j in self.j_values:
        #     tmp  = []
        #     for i in range(self.M):
        #         tmp.append(self.grid[i, j])
        #         pass
        #     possible_prices[j] = tmp
        #     pass
        
        print("hello")
        price_key = {}
        for i in range(self.M):
            tmp  = []
            for j in self.j_values:
                tmp.append(self.grid[i, j])
                pass
            price_key[i] = tmp
            pass
            
        return price_key
    
    def get_info(self):
        infos = {}
        for j in self.j_values:
            tmp  = []
            for i in range(self.M)[:]:
                info = {'asset': i*self.dS, 'time': j*self.dt, 'option': self.grid[i, j]}
                tmp.append(info)
                pass
            infos[j] = tmp
        return infos

# The Explicit Method
## Difference approximation method
- Forward Difference approximation with respect to t.
- First-order Central Difference approximation with respect to S.
- Second-Order Symmetric Central Difference approximation with respect to S.
- To avoid instability issues, we need to set a reasonable intervals of time t, where 
            $0 < \Delta t < \frac{1}{\sigma^2(M-1) + \frac{1}{2}r}$.

In [3]:
""" Explicit method of Finite Differences using forward differenciation for time """
class FDExplicitEu(FiniteDifferences):

    def _setup_boundary_conditions_(self):
        if self.is_call:
            # Initial condition: At the beginning of the option contract, the price of the option should be equal to 
            self.grid[:, 0] = np.maximum(self.boundary_conds - self.K, 0)
            
            # Boundary condition: Terminal when the asset price hit the max value
            self.grid[-1, 1:] = (self.Smax - self.K) * np.exp(-self.r *
                                                               self.dt *
                                                               (self.j_values))
        else:
            # Initial condition: 
            self.grid[:, 0] = np.maximum(self.K - self.boundary_conds, 0)
            
            # Boundary condition: Terminal when the asset price hit the max value
            self.grid[0, 1:] = (self.K - self.Smax) * np.exp(-self.r * 
                                                              self.dt * 
                                                              (self.j_values))

    def _setup_coefficients_(self):
        self.a = 0.5 * self.dt * ((self.sigma**2) *
                                  (self.i_values**2) - 
                                  self.r * self.i_values)
        self.b = 1 - self.dt * ((self.sigma**2) * 
                                (self.i_values**2) + 
                                self.r)
        self.c = 0.5 * self.dt * ((self.sigma**2) * 
                                  (self.i_values**2) + 
                                  self.r*self.i_values)

    def _traverse_grid_(self):
        for j in self.j_values[:]:
            for i in range(self.M)[:]:
                self.grid[i, j+1] = self.a[i] * self.grid[i-1, j] + self.b[i] * self.grid[i, j] + self.c[i] * self.grid[i+1, j] 